In [9]:
import pickle
import time
start_time = time.time()
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import SimpleITK as sitk
import json
import ProstateLesionDetectionUtils
import random
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import ndimage
import math
import tensorflow_addons as tfa
import tensorflow as tf

with open("ProstateNetData/Datasets.json", 'r') as f:
    datasets = json.load(f)
for dataset in datasets.keys():
    for pat in datasets[dataset].keys():
        for seq in datasets[dataset][pat].keys():
            x = datasets[dataset][pat][seq]
            x = x.replace("\\","/")
            datasets[dataset][pat][seq] = x
# p158 = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATE158\DATASETPICKLE\Nifti_files"
# picai = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PICAI_DATA\Nifti_files"
prostatenet = "ProstateNetData/Dataset011_ProstateNet"
with open("ProstateNetData/folds.json", 'r') as f:
    ValidationFolds = json.load(f)
    

def PrepareTraining(pat):
    dataset, labels = {}, {}
    for key in pat.keys():
        dataset.update({key:np.transpose(np.stack([pat[key]["T2"], pat[key]["ADC"], pat[key]["DWI"]]),(1,2,3,0))}) # tf.convert_to_tensor
        labels.update({key:np.expand_dims(pat[key]["Lesions"], axis = -1)}) # tf.convert_to_tensor
    return dataset, labels

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
loader = ProstateLesionDetectionUtils.DatasetsLoaders.Nifti_Loader.LoadSitkImages(ptdc = datasets["ProstateNet"], folder = prostatenet)
patients = loader.load_npobj()

In [ ]:
vis = ProstateLesionDetectionUtils.DatasetsProcessing.Visualization.Visualizer(patients)
vis.np_vis("DWI", "PCa-324365095643606010864476831387746614101")

In [ ]:
dt = ProstateLesionDetectionUtils.DetectionMetrics.ModelDetoriention.DetorientLesions(patients, rates = [15,40,50])
dt.CreateZones()
dt.CreateZaxisZone()
dt.KeepZonesNonZeroImageValues()
zones = dt.GetZones()
zaxis = dt.GetZaxisZones()

In [4]:
dataset, labels = PrepareTraining(patients)

x_train = {key:dataset[key] for key in dataset.keys() if key not in ValidationFolds["Fold 0"]}
y_train = {key:labels[key] for key in labels.keys() if key not in ValidationFolds["Fold 0"]}

x_test = {key:dataset[key] for key in dataset.keys() if key in ValidationFolds["Fold 0"]}
y_test = {key:labels[key] for key in labels.keys() if key in ValidationFolds["Fold 0"]}

In [5]:
len(x_train),len(y_train),len(x_test),len(y_test)

(289, 289, 73, 73)

In [6]:
Xtrain = np.array(list(x_train.values()))
Ytrain = np.array(list(y_train.values()))
Xtest = np.array(list(x_test.values()))
Ytest = np.array(list(y_test.values()))

In [7]:
start_time = time.time()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
FILTERS = [32, 64, 128, 256, 512, 1024, 2048]
Pool_size = ((2,2,2),(2,2,2),(2,2,2), (1,2,2), (1,2,2), (1,2,2))
VOLSIZE = 24
INPUT_SIZE = (1,24,192,192,3)
BATCH_SIZE=1
EPOCHS = 400
INIT_LR = 1e-5
MAX_LR = 1e-2
TQDM_CB=tfa.callbacks.TQDMProgressBar()

METRICS = "acc" # tfa.metrics.F1Score(num_classes=2, threshold=0.5)

STEPS_PER_EPOCH = len(Xtrain) // BATCH_SIZE # len(train_data)

# CLR = lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     0.001,
#     decay_steps=STEPS_PER_EPOCH,  # Usually set to total number of training steps/epochs
#     decay_rate=0.96,
#     staircase=True
#)
CLR = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
    maximal_learning_rate=MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * STEPS_PER_EPOCH)
LOSS = tfa.losses.sigmoid_focal_crossentropy  #tfa.losses.sigmoid_focal_crossentropy
OPTIMIZER = tf.keras.optimizers.legacy.Adam(CLR)

# name_folder = "TransUnet_8translayers_8heads"
# try:
#     os.mkdir(os.path.join(r"E:\DIMITRIS_DEEP_LEARNING\RESULTS\PROCANCER_UC1_RESULTS",name_folder))
# except:
#     pass

# CHECK_CB = tf.keras.callbacks.ModelCheckpoint(os.path.join(r"E:\DIMITRIS_DEEP_LEARNING\RESULTS\PROCANCER_UC1_RESULTS",name_folder+"\CheckPoint_Weights.tf"),save_weights_only=False,save_best_only=True)
# #EARLY_STOP = tf.keras.callbacks.EarlyStopping(patience = 30)
# CALLBACKS=[CHECK_CB,TQDM_CB]#
# #train_dataset = tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain))
# #val_dataset = tf.data.Dataset.from_tensor_slices((Xtest, Ytest))


# # train_dataset = train_dataset.batch(BATCH_SIZE)
# # val_dataset = val_dataset.batch(BATCH_SIZE)

# TrainParams = {"LOSS":LOSS, "OPTIMIZER":OPTIMIZER, "CALLBACKS":CALLBACKS,"POOL_SIZE":Pool_size,
#                "BATCH_SIZE":BATCH_SIZE, "FILTERS":FILTERS, "INPUT_SIZE":INPSIZE, "METRICS":METRICS,
#                "WEIGHTS":None,
#                "VOLUME_SIZE":24, "EPOCHS":EPOCHS, "TRAIN_DATASET":{"DATA":Xtrain,"LABELS":Ytrain},"VAL_DATASET":{"DATA":Xtest,"LABELS":Ytest},"DENSE_SIZE":16, "PATCH_SIZE_XY": 3,
#                 "PATCH_SIZE_Z": 1,
#                 "TRANSF_HEADS": 8,
#                 "TRANSF_LAYERS": 8,
#                 "DROP_RATE": 0.3}

# print((time.time() - start_time))

/home/medadmin/anaconda3/envs/tf12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
models = ['Unet', 'Attention_Unet', 'Vnet', 'USE_net', 'Dense2_Unet', 'Trans_Unet']
mod_instances = [
ProstateLesionDetectionUtils.DetectionModels.Models3D.UnetModel3D.UNET.TrainUnet,
ProstateLesionDetectionUtils.DetectionModels.Models3D.AttentionUNet3D.AttentionUNet.TrainAttentionUnet,
ProstateLesionDetectionUtils.DetectionModels.Models3D.VNet.vnet.TrainVnet,
ProstateLesionDetectionUtils.DetectionModels.Models3D.USENet3D.USENET.TrainUSEnet,
ProstateLesionDetectionUtils.DetectionModels.Models3D.Dense2VNet.D2Vnet.TrainDenseVnet,
ProstateLesionDetectionUtils.DetectionModels.Models3D.TransUnet3D.transunet3d.TrainTransUnet,
]

TrainParams = {"LOSS":LOSS, "OPTIMIZER":OPTIMIZER, "CALLBACKS":None,"POOL_SIZE":Pool_size,
            "BATCH_SIZE":BATCH_SIZE, "FILTERS":FILTERS, "INPUT_SIZE":INPUT_SIZE, "METRICS":METRICS,
            "INITIAL_WEIGHTS":None,
            "WEIGHTS":None,
            "VOLUME_SIZE":24, "EPOCHS":EPOCHS, 
            "TRAIN_DATASET":{"DATA":Xtrain,"LABELS":Ytrain},"VAL_DATASET":{"DATA":Xtest,"LABELS":Ytest},
            "DENSE_SIZE":32, "GROWTH_RATE": 2, "DROPOUT_RATE": 0.3, "DENSE_BLOCKS": 2,
            "PATCH_SIZE_XY": 3,
            "PATCH_SIZE_Z": 1,
            "TRANSF_HEADS": 8,
            "TRANSF_LAYERS": 8,
            "DROP_RATE": 0.3}

for mdl_pth, mdl in zip(models,mod_instances):
    if mdl_pth == "Dense2_Unet" or mdl_pth == "Trans_Unet":
        pth_check = ["EXPERIMENTS",mdl_pth,"fold_0","Checkpoint_2048.tf"]
        pth_final = ["EXPERIMENTS",mdl_pth,"fold_0","Final_2048.tf"]
        pth_init = ["EXPERIMENTS",mdl_pth,"init_weights_2048.h5"]

        CHECK_CB = tf.keras.callbacks.ModelCheckpoint(os.path.join(*pth_check),save_weights_only=False,save_best_only=True)
        CALLBACKS=[CHECK_CB]#
        TrainParams["CALLBACKS"] = CALLBACKS
        TrainParams["INITIAL_WEIGHTS"] = os.path.join(*pth_init)
        TrainParams["WEIGHTS"] = os.path.join(*pth_final)

        mod = mdl(TrainParams)
        mod.ModelBuild()

        if "init_weights_2048.h5" in os.listdir(os.path.join("EXPERIMENTS",mdl_pth)):
            mod.LoadTrained()
        else:
            mod.SaveInitWeights()
        mod.LoadTrained()
        mod.ModelCompile()
        mod.ModelFit()

        mod = mod.GetModel()
        mod.save(TrainParams["WEIGHTS"])

Epoch 1/400
289/289 [==============================] - ETA: 0s - loss: 0.9560 - acc: 0.9143

289/289 [==============================] - 263s 824ms/step - loss: 0.9560 - acc: 0.9143 - val_loss: 0.2711 - val_acc: 0.9983
Epoch 2/400
289/289 [==============================] - ETA: 0s - loss: 0.0622 - acc: 0.9955

289/289 [==============================] - 242s 838ms/step - loss: 0.0622 - acc: 0.9955 - val_loss: 0.0061 - val_acc: 0.9983
Epoch 3/400
289/289 [==============================] - ETA: 0s - loss: 0.0049 - acc: 0.9955

289/289 [==============================] - 242s 838ms/step - loss: 0.0049 - acc: 0.9955 - val_loss: 0.0031 - val_acc: 0.9983
Epoch 4/400
289/289 [==============================] - ETA: 0s - loss: 0.0036 - acc: 0.9955

289/289 [==============================] - 245s 849ms/step - loss: 0.0036 - acc: 0.9955 - val_loss: 0.0026 - val_acc: 0.9983
Epoch 5/400
289/289 [==============================] - 194s 671ms/step - loss: 0.0035 - acc: 0.9955 - val_loss: 0.0038 - val_acc: 0.9983
Epoch 6/400
289/289 [==============================] - ETA: 0s - loss: 0.0034 - acc: 0.9955

...

289/289 [==============================] - 249s 864ms/step - loss: 0.0046 - acc: 0.9999 - val_loss: 0.0321 - val_acc: 0.9827
Epoch 333/400
289/289 [==============================] - 170s 587ms/step - loss: 0.0043 - acc: 0.9999 - val_loss: 0.0434 - val_acc: 0.9792
Epoch 334/400
289/289 [==============================] - 170s 588ms/step - loss: 0.0040 - acc: 0.9999 - val_loss: 0.0357 - val_acc: 0.9812
Epoch 335/400
289/289 [==============================] - 167s 579ms/step - loss: 0.0037 - acc: 0.9998 - val_loss: 0.0406 - val_acc: 0.9785
Epoch 336/400
289/289 [==============================] - ETA: 0s - loss: 0.0034 - acc: 0.9998

289/289 [==============================] - 252s 871ms/step - loss: 0.0034 - acc: 0.9998 - val_loss: 0.0278 - val_acc: 0.9836
Epoch 337/400
289/289 [==============================] - ETA: 0s - loss: 0.0032 - acc: 0.9998

289/289 [==============================] - 252s 873ms/step - loss: 0.0032 - acc: 0.9998 - val_loss: 0.0240 - val_acc: 0.9842
Epoch 338/400
289/289 [==============================] - ETA: 0s - loss: 0.0029 - acc: 0.9998

289/289 [==============================] - 250s 867ms/step - loss: 0.0029 - acc: 0.9998 - val_loss: 0.0227 - val_acc: 0.9847
Epoch 339/400
289/289 [==============================] - 167s 578ms/step - loss: 0.0027 - acc: 0.9998 - val_loss: 0.0281 - val_acc: 0.9821
Epoch 340/400
289/289 [==============================] - 167s 577ms/step - loss: 0.0025 - acc: 0.9998 - val_loss: 0.0384 - val_acc: 0.9789
Epoch 341/400
289/289 [==============================] - 168s 581ms/step - loss: 0.0023 - acc: 0.9998 - val_loss: 0.0384 - val_acc: 0.9789
Epoch 342/400
289/289 [==============================] - 167s 577ms/step - loss: 0.0022 - acc: 0.9998 - val_loss: 0.0287 - val_acc: 0.9825
Epoch 343/400
289/289 [==============================] - 165s 572ms/step - loss: 0.0020 - acc: 0.9998 - val_loss: 0.0305 - val_acc: 0.9807
Epoch 344/400
289/289 [==============================] - ETA: 0s - loss: 0.0019 - acc: 0.9997

289/289 [==============================] - 252s 874ms/step - loss: 0.0019 - acc: 0.9997 - val_loss: 0.0227 - val_acc: 0.9834
Epoch 345/400
289/289 [==============================] - 168s 581ms/step - loss: 0.0018 - acc: 0.9997 - val_loss: 0.0270 - val_acc: 0.9814
Epoch 346/400
289/289 [==============================] - 169s 584ms/step - loss: 0.0017 - acc: 0.9997 - val_loss: 0.0267 - val_acc: 0.9814
Epoch 347/400
289/289 [==============================] - 167s 579ms/step - loss: 0.0016 - acc: 0.9997 - val_loss: 0.0328 - val_acc: 0.9794
Epoch 348/400
289/289 [==============================] - 167s 576ms/step - loss: 0.0015 - acc: 0.9997 - val_loss: 0.0398 - val_acc: 0.9774
Epoch 349/400
289/289 [==============================] - 168s 580ms/step - loss: 0.0015 - acc: 0.9995 - val_loss: 0.0269 - val_acc: 0.9799
Epoch 350/400
289/289 [==============================] - 167s 577ms/step - loss: 0.0015 - acc: 0.9990 - val_loss: 0.0229 - val_acc: 0.9788
Epoch 351/400
289/289 [==================